<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/MLOps_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 수집

In [1]:
!pip install -q tfx==1.2.0

     |████████████████████████████████| 2.4 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 
     |████████████████████████████████| 1.7 MB 57.9 MB/s 
     |████████████████████████████████| 189 kB 42.9 MB/s 
     |████████████████████████████████| 454.4 MB 16 kB/s 
     |████████████████████████████████| 17.7 MB 239 kB/s 
     |████████████████████████████████| 9.8 MB 39.2 MB/s 
     |████████████████████████████████| 1.7 MB 38.8 MB/s 
     |████████████████████████████████| 1.3 MB 53.1 MB/s 
     |████████████████████████████████| 40 kB 4.4 MB/s 
     |████████████████████████████████| 1.4 MB 39.8 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 147 kB 47.7 MB/s 
     |████████████████████████████████| 135 kB 37.5 MB/s 
     |████████████████████████████████| 406 kB 39.4 MB/s 
     |████████████████████████████████| 49 kB 4.3 MB/s 
     |████████████████████████████████| 6.5 MB 43.5 MB/s 
     |███████████████

In [2]:
!pip install google-apitools==0.5.26

     |████████████████████████████████| 238 kB 5.2 MB/s 
  Created wheel for google-apitools: filename=google_apitools-0.5.26-py3-none-any.whl size=129455 sha256=59504ed92c7245d84cb36ca4f278022d7b26cae1026e1b9065c62ec37c17a8d0
  Stored in directory: /root/.cache/pip/wheels/c9/33/97/e3d85ff94f31ed99b9f4389f452f1be1dfe8424c02e06eb482
Successfully built google-apitools
  Attempting uninstall: google-apitools
    Found existing installation: google-apitools 0.5.31
    Uninstalling google-apitools-0.5.31:
      Successfully uninstalled google-apitools-0.5.31


In [3]:
import tensorflow as tf

with tf.io.TFRecordWriter('test.tfrecord') as w:
  w.write(b'First record')
  w.write(b'Second record')

for record in tf.data.TFRecordDataset('test.tfrecord'):
  print(record)

tf.Tensor(b'First record', shape=(), dtype=string)
tf.Tensor(b'Second record', shape=(), dtype=string)


In [4]:
import os
from pathlib import Path
from tfx.components import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

dir_path = os.getcwd()

# 데이터 경로 정의
input_base = os.path.join(dir_path, "..", "..", "data", "taxi")
# 파이프라인 컴포넌트 인스턴스화
example_gen = CsvExampleGen(input_base=input_base)
# 대화형으로 컴포넌트 실행
context.run(example_gen)

FileNotFoundError: ignored

In [5]:
import os
from pathlib import Path
from tfx.components import ImportExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

dir_path = os.getcwd()
tfrecord_dir = os.path.join(dir_path, "..", "..", "data", "tfrecord_data")

example_gen = ImportExampleGen(input_base=tfrecord_dir)

context.run(example_gen)

FileNotFoundError: ignored

In [6]:
from tfx.components.base import executor_spec

# 일반 파일 로더 컴포넌트
from tfx.components import FileBasedExampleGen

# Parquet 관련 실행자
from tfx.components.example_gen.custom_executors import parquet_executor

parquet_dir_path = 'parquet_data'

# Executor 재정의
example_gen = FileBasedExampleGen(
    input_base=parquet_dir_path,
    custom_executor_spec=executor_spec.ExecutorClassSpec(
        parquet_executor.Executor
  )
)

FileNotFoundError: ignored

In [7]:
from tfx.components.base import executor_spec

from tfx.components import FileBasedExampleGen

# Avro 관련 실행자
from tfx.components.example_gen.custom_executors import avro_executor

avro_dir_path = 'avro_data'

example_gen = FileBasedExampleGen(
    input_base=avro_dir_path,
    custom_executor_spec=executor_spec.ExecutorClassSpec(
        avro_executor.Executor
  )
)

FileNotFoundError: ignored

In [8]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
from pathlib import Path

# 원시 데이터 다운로드
filepath = tf.keras.utils.get_file(
    'complaints.csv.zip',
    'http://files.consumerfinance.gov/ccdb/complaints.csv.zip'
)

dir_path = os.getcwd()
data_dir = os.path.join(dir_path, "..", "..", "data")
processed_dir = os.path.join(dir_path, "..", "..", "data", "processed")
Path(processed_dir).mkdir(parents=True, exist_ok=True)

# 압축 해제
shutil.unpack_archive(filepath, data_dir)

df = pd.read_csv(os.path.join(data_dir, 'complaints.csv'))

df.columns = [
              'datae_recieved', 'product', 'sub_product', 'issue', 'sub_issue',
              'consumer_complaint_narrative', 'company_public_response',
              'company', 'state', 'zip_code', 'tags',
              'consumer_consent_provided', 'submitted_via',
              'date_sent_to_company', 'company_response',
              'timely_response', 'consumer_disputed', 'complaint_id'
]

df.loc[df['consumer_disputed'] == "", "consumer_disputed"] = np.nan

# 주요 필드 결측치일 경우 레코드 제외
df = df.dropna(subset=['consumer_complaint_narrative', 'consumer_disputed'])

# consumer_disputed의 Yes와 No를 1과 0으로 변경
df.loc[df['consumer_disputed'] == 'Yes', 'consumer_disputed'] = 1
df.loc[df['consumer_disputed'] == 'No', 'consumer_disputed'] = 0

# zip_code
df.loc[df['zip_code'] == "", "zip_code"] = '000000'
df.loc[pd.isna(df['zip_code']), 'zip_code'] = '000000'

df = df[df['zip_code'].str.len() == 5]
df['zip_code'] = df['zip_code'].str.replace("XX", "00")
df = df.reset_index(drop=True)
df['zip_code'] = pd.to_numeric(df['zip_code'], errors='coerce')

# dataframe to csv
df.to_csv(os.path.join(processed_dir, 'processed-complaints.csv'), index=False)

395386880/395381402 [==============================] - 9s 0us/step


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  #-------------------------------------------------------------------------
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [18]:
import os
import re

import tensorflow as tf
import pandas as pd

# helper functions
def _bytes_feature(value):
  return tf.train.Feature(
      bytes_list=tf.train.BytesList(value=[value.encode()])
      )
  
def _float_feature(value):
  return tf.train.Feature(
      float_list=tf.train.FloatList(value=[value])
      )

def _int64_feature(value):
  return tf.train.Feature(
      int64_list=tf.train.Int64List(value=[value])
      )

def clean_rows(row):
  if pd.isna(row['zip_code']):
    row['zip_code'] = '99999'
  return row

def convert_zipcode_to_int(zipcode):
  nums = re.findall(r'\d+', zipcode)
  if len(nums) > 0:
    int_zipcode = int(nums[0])
  else:
    int_zipcode = 99999
  return int_zipcode

# path
dir_path = os.getcwd()
data_dir = os.path.join(dir_path, "..", "..", "data")
tfrecord_dir = os.path.join(dir_path, "..", "..", "data", "tfrecord")

df = pd.read_csv(os.path.join(data_dir, 'processed/processed-complaints.csv'))

Path(tfrecord_dir).mkdir(parents=True, exist_ok=True)

tfrecord_filename = 'consumer-complaints.tfrecord'
tfrecord_filepath = os.path.join(tfrecord_dir, tfrecord_filename)

# TFRecord 객체 생성
tf_record_writer = tf.io.TFRecordWriter(tfrecord_filepath)

for idx, row in df.iterrows():
  row = clean_rows(row)

  # 모든 데이터 레코드를 tf.train.Example로 변환
  example = tf.train.Example(
      features=tf.train.Features(
          feature={
              'product': _bytes_feature(str(row['product'])),
              'sub_product': _bytes_feature(str(row['sub_product'])),
              'issue': _bytes_feature(str(row['issue'])),
              'sub_issue': _bytes_feature(str(row['sub_issue'])),
              'state': _bytes_feature(str(row['state'])),
              'zip_code': _int64_feature(convert_zipcode_to_int(str(row['zip_code']))),
              'company': _bytes_feature(str(row['company'])),
              'company_response': _bytes_feature(str(row['company_response'])),
              'timely_response': _bytes_feature(str(row['timely_response'])),
              'consumer_disputed': _float_feature(row['consumer_disputed']),
          }
      )
  )

  # 데이터 구조 직렬화
  tf_record_writer.write(example.SerializeToString())
tf_record_writer.close()